In [5]:
import os, sys

sys.path.append(os.path.dirname(os.path.realpath("../parser")))
sys.path.append(os.path.dirname(os.path.realpath("../feed_handler")))

In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

In [7]:
from parser.MAV.models import DeclarativeBase

DeclarativeBase.metadata.create_all(engine)

2022-11-09 17:29:38,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:29:38,845 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("RSS_events_MAV")
2022-11-09 17:29:38,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:29:38,851 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("RSS_events_MAV")
2022-11-09 17:29:38,853 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:29:38,855 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("RAW_events_MAV")
2022-11-09 17:29:38,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:29:38,860 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("RAW_events_MAV")
2022-11-09 17:29:38,867 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-09 17:29:38,874 INFO sqlalchemy.engine.Engine 
CREATE TABLE "RSS_events_MAV" (
	uuid BINARY(16) NOT NULL, 
	id INTEGER NOT NULL, 
	last_modification DATE NOT NULL, 
	update_uuid BINARY(16), 
	title VARCHAR(50), 
	PRIMARY KEY (uuid), 
	UNIQUE (uuid)
)


2022-11-09 17:29:38,

In [8]:
from feed_handler.feed_consumer import FeedConsumer
from datetime import datetime
from time import mktime

Feed = FeedConsumer()
d = Feed.fetch()
time_struct = d.entries[0]["published_parsed"]
date = datetime.fromtimestamp(mktime(time_struct))
print(date)

2022-11-09 16:10:31


In [9]:
from sqlalchemy.orm import Session
from parser.MAV.feed_parser import FeedParser

with Session(engine) as session:
    entry = FeedParser.parse_entry(d.entries[0])
    entry_dump = FeedParser.dump_entry(d.entries[0], entry.uuid)
    session.add_all([entry, entry_dump])
    session.commit()

2022-11-09 17:29:39,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:29:39,395 INFO sqlalchemy.engine.Engine INSERT INTO "RSS_events_MAV" (uuid, id, last_modification, update_uuid, title) VALUES (?, ?, ?, ?, ?)
2022-11-09 17:29:39,398 INFO sqlalchemy.engine.Engine [generated in 0.00325s] (<memory at 0x000001F1544350C0>, '113333', '2022-11-09', None, 'Kimarad két délutáni S210-es vonat')
2022-11-09 17:29:39,403 INFO sqlalchemy.engine.Engine INSERT INTO "RAW_events_MAV" (event_uuid, event_dict) VALUES (?, ?)
2022-11-09 17:29:39,404 INFO sqlalchemy.engine.Engine [generated in 0.00184s] (None, '{"title": "Kimarad k\\u00e9t d\\u00e9lut\\u00e1ni S210-es vonat", "title_detail": {"type": "text/plain", "language": "hu", "base": "https://www.mavcs ... (568 characters truncated) ... : "M\\u00c1V.hu szerkeszt\\u0151s\\u00e9g", "author_detail": {"name": "M\\u00c1V.hu szerkeszt\\u0151s\\u00e9g"}, "id": "113333", "guidislink": false}')
2022-11-09 17:29:39,408 INFO sqlalchemy.engine.En

In [10]:
from sqlalchemy import select
from parser.MAV.models import MAVEvent

with Session(engine) as session:
    events = session.query(MAVEvent)
    print(events.first().id)

2022-11-09 17:29:39,782 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-09 17:29:39,794 INFO sqlalchemy.engine.Engine SELECT "RSS_events_MAV".uuid AS "RSS_events_MAV_uuid", "RSS_events_MAV".id AS "RSS_events_MAV_id", "RSS_events_MAV".last_modification AS "RSS_events_MAV_last_modification", "RSS_events_MAV".update_uuid AS "RSS_events_MAV_update_uuid", "RSS_events_MAV".title AS "RSS_events_MAV_title" 
FROM "RSS_events_MAV"
 LIMIT ? OFFSET ?
2022-11-09 17:29:39,796 INFO sqlalchemy.engine.Engine [generated in 0.00223s] (1, 0)
113333
2022-11-09 17:29:39,801 INFO sqlalchemy.engine.Engine ROLLBACK
